<a href="https://colab.research.google.com/github/dvlpana/SparkPlatzi/blob/main/CursoSpakPlatzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!apt-get install  openjdk-8-jdk-headless -qq > /dev/null
!ad-apt-repository ppa:openjdk-r/ppa
!apt-get update && sudo apt-get upgrade -y
!apt-get -y install openjdk-8-jre
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

/bin/bash: ad-apt-repository: command not found
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13

In [3]:
!pip install -q findspark

In [4]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7  spark-3.1.1-bin-hadoop2.7.tgz


In [5]:
import time
print("Sleeping")
time.sleep(30) # sleep for a while; interrupt me!
print("Done Sleeping")

Sleeping
Done Sleeping


In [6]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7  spark-3.1.1-bin-hadoop2.7.tgz


In [7]:
!head -n 15 /content/sample_data/files_curso_spark/data.csv

State,Color,Count
TX,Red,20
NV,Blue,66
CO,Blue,79
OR,Blue,71
WA,Yellow,93
WY,Blue,16
CA,Yellow,53
WA,Green,60
OR,Green,71
TX,Green,68
NV,Green,59
AZ,Brown,95
WA,Yellow,20
AZ,Blue,75


In [8]:
!spark-submit /content/sample_data/files_curso_spark/codeExample.py /content/sample_data/files_curso_spark/data.csv

21/06/08 16:05:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/06/08 16:05:29 INFO SparkContext: Running Spark version 3.1.2
21/06/08 16:05:29 INFO ResourceUtils: ==============================================================
21/06/08 16:05:29 INFO ResourceUtils: No custom resources configured for spark.driver.
21/06/08 16:05:29 INFO ResourceUtils: ==============================================================
21/06/08 16:05:29 INFO SparkContext: Submitted application: PythonMnMCount
21/06/08 16:05:29 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
21/06/08 16:05:29 INFO Resour

In [9]:
!ls

drive  sample_data  spark-3.1.1-bin-hadoop2.7  spark-3.1.1-bin-hadoop2.7.tgz


In [10]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [11]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [12]:
archivo = './sample_data/files_curso_spark/data.csv'
df_spark = spark.read.csv(archivo, inferSchema=True, header=True)

# imprimir tipo de archivo
print(type(df_spark))

<class 'pyspark.sql.dataframe.DataFrame'>


¿Nombre de las Columnas de dataframe?

In [13]:
df_spark.columns

['State', 'Color', 'Count']

Estructura del dataframe

In [14]:
df_spark.printSchema()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)



Ver los primeros 20 registros del dataframe

In [15]:
df_spark.show()

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   WY|  Blue|   16|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
|   NV| Green|   59|
|   AZ| Brown|   95|
|   WA|Yellow|   20|
|   AZ|  Blue|   75|
|   OR| Brown|   72|
|   NV|   Red|   98|
|   WY|Orange|   45|
|   CO|  Blue|   52|
|   TX| Brown|   94|
|   CO|   Red|   82|
+-----+------+-----+
only showing top 20 rows



Descricipcion Estadistica del dataframe

In [16]:
df_spark.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
State,99999,None,None,AZ,WY
Color,99999,None,None,Blue,Yellow
Count,99999,55.00090000900009,26.26648202084822,10,100


Descripcion estadistica de una sola columna (‘median_house_value’)

In [17]:
df_spark.describe(['color']).show()

+-------+------+
|summary| color|
+-------+------+
|  count| 99999|
|   mean|  null|
| stddev|  null|
|    min|  Blue|
|    max|Yellow|
+-------+------+



## 8 Transformaciones y Acciones

In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext
lines = sc.textFile("spark-2.4.4-bin-hadoop2.7/README.md")
sc



<SparkContext master=local[*] appName=pyspark-shell>

In [19]:
path = '/content/sample_data/files_curso_spark/'
equiposOlimpicosRDD = sc.textFile(path+'paises.csv') \
.map(lambda line: line.split(","))

# Muestra la data con take , evita utilizar collect()

In [20]:
equiposOlimpicosRDD.take(10)



[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

9 Acciones de modificacion sobre RDDs

In [21]:
##Contar cuantas siglas tenemos
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [22]:
##Agrupación a partir de siglas y obtener cantidad de equipos por país
equiposOlimpicosRDD \
.map(lambda x: (x[2], x[1])) \
.groupByKey() \
.mapValues(len).take(5)

[('ARG', 18), ('AFG', 1), ('IRL', 7), ('POR', 21), ('AUS', 23)]

In [23]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [24]:
##Agrupación a partir de siglas y obtener cantidad de equipos por país
equiposOlimpicosRDD \
.map(lambda x: (x[2], x[1])) \
.groupByKey() \
.mapValues(list).take(3)

[('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan']),
 ('IRL',
  ['Akatonbo',
   'Aletta',
   'Ireland',
   'Ireland-1',
   'Ireland-2',
   'The Cloud',
   'Three Leaves'])]

In [25]:
##Obtener los datos sólo de un país o contenido con filter()
equiposArgentinos = equiposOlimpicosRDD.filter(lambda x: "ARG" in x)
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [26]:
## Cuenta todos los valores, nada recomendable para datos extensos
equiposOlimpicosRDD.count()

1185

In [27]:
##Cuenta  hasta el tiempo aproximado
equiposOlimpicosRDD.countApprox(20)

1185

10 Acciones de conteo sobre RDDs

In [28]:
## importamos ambos
deportistaOlimpicoRDD = sc.textFile(path+'deportista.csv') \
.map(lambda line: line.split(","))

deportistaOlimpicoRDD2 = sc.textFile(path+'deportista2.csv') \
.map(lambda line: line.split(","))

In [30]:
## Unimos los RDD con union(). Spark solo tiene esta operacion
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

Py4JJavaError: ignored

In [ ]:
deportistaOlimpicoRDD.count()

In [ ]:
##Ver contenido en equipos olimpicos
equiposOlimpicosRDD.top(2)

In [ ]:
deportistaOlimpicoRDD.top(2)

In [ ]:
# seleccionamos las comumnas ID para hacer un join
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.take(6)

In [ ]:
# utilizamos takeSample (repetidos, muestra, semilla)
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.takeSample(False,6,25)

In [ ]:
resultado = sc.textFile(path+'resultados.csv') \
.map(lambda line: line.split(","))

In [ ]:
resultadoGanador = resultado.filter(lambda w: 'NA' not in w[1])

In [ ]:
resultadoGanador.take(5)

In [ ]:
##Reto Hacer un join con equipos y deportista para obtener valores importantes
deportistaOlimpicoRDD \
.map(lambda deportista: [deportista[-1],deportista[:-1]]) \
.join(equiposOlimpicosRDD.map(lambda equipo: [equipo[0], equipo [2]])) \
.join(resultadoGanador.map(lambda resultado: [resultado[2], resultado [1]])) \
.takeSample(False, 6, 25)


Clase 11 Solucion reto deportistas

In [ ]:
#esta clase muestra la solucion del profesor al reto.
deportistaPaises = deportistaOlimpicoRDD \
    .map(lambda l: [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]]))

deportistaPaises.join(resultadoGanador).take(6)


#Clase 12 Operaciones Numéricas
##En esta clase hacemos el encoding de los valores de las medallas y obtenemos los puntos que tiene cada país.
Acciones de Conteo sobre RDD

```
# Tiene formato de código
```



In [ ]:
!ls ./sample_data/files_curso_spark/

In [ ]:
#Almacenamos el puntaje por medalla
valoresMedallas = {'Gold'  :7,
                   'Silver':5,
                   'Bronze':4}

In [ ]:
paisesMedallas = deportistaPaises.join(resultadoGanador)



In [ ]:
paisesMedallas = paisesMedallas \
    .map(lambda x: (x[1][0][-1], valoresMedallas[x[1][1] ] ) )

In [ ]:
paisesMedallas.take(5)

In [ ]:
from operator import add

conclusion = paisesMedallas.reduceByKey((add)) \
    .sortBy(lambda x:x[1], ascending=False)

In [ ]:
conclusion.take(10)

Clase 13 Creación de DataFrames

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row

from pyspark.sql import SQLContext





In [ ]:
#spark = SparkContext(master='local', appName="DataFrames")
sqlContext = SQLContext(spark)




In [ ]:
!ls ./sample_data/files_curso_spark/

In [ ]:
!head -n 10 ./sample_data/files_curso_spark/juegos.csv

In [ ]:
spark

In [ ]:
# Paso 1 crear un dataFrame es crearun Schema como una base de datos
path = '/content/sample_data/files_curso_spark/'
juegoSchema = StructType([
     StructField('juego_id', IntegerType(), False),
     StructField('anio', StringType(), False),
     StructField('temporada', StringType(), False),
     StructField('ciudad', StringType(), False),
    ])
# Paso 2 Crea Data Frame con la Lectura del Schema 
juegoDF = sqlContext.read.schema(juegoSchema) \
          .option('header', 'true').csv(path+'resultados.csv')

In [ ]:
juegoDF.show(4)

Clase 14 Inferencia de tipos de datos

In [ ]:
deportistaOlimpicoRDD.take(5)

In [ ]:
# Retiramos encabezados
def eliminarEncabezado(indice, iterador):
  return iter(list(iterador)[1:])

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.\
mapPartitionsWithIndex(eliminarEncabezado)

In [ ]:
deportistaOlimpicoRDD.take(5)

In [ ]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6]),
))

In [ ]:
deportistaOlimpicoRDD.take(5)

In [ ]:
schema_deportistaOlimpico = StructType([
    StructField('deportista_id', IntegerType(),False),
    StructField('nombre', StringType(),False),
    StructField('genero', IntegerType(),False),
    StructField('edad', IntegerType(),False),
    StructField('altura', IntegerType(),False),
    StructField('peso', FloatType(),False),
    StructField('equipo_id', IntegerType(),False)
]    
)

In [ ]:
deportistaDF =  sqlContext.createDataFrame(deportistaOlimpicoRDD, schema_deportistaOlimpico)

In [ ]:
deportistaDF.show(5)

Retos crear los demas DataFrames testantes a partir de los RDD

In [ ]:
# Países  equipos, sc???
paisesRDD = sc.textFile(path+'paises.csv') \
.map(lambda line: line.split(",")) \
.mapPartitionsWithIndex(eliminarEncabezado) \
.map(lambda l:(
    int(l[0]),
    l[1],
    l[2]
))

schema = StructType([
    StructField('pais_id', IntegerType(),False),
    StructField('equipo', StringType(),False),
    StructField('sigla', StringType(),False)

]    
)

paisesDF =  sqlContext.createDataFrame(paisesRDD, schema)
paisesDF.show(5)

In [ ]:
resultados_schema = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False)
])

resultadosDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(resultados_schema).\
            load(path+"resultados.csv")

resultadosDF.show(4)

In [ ]:
evento_schema = StructType([
    StructField("evento_id", IntegerType(), False),
    StructField("evento", StringType(), False), 
    StructField("deporte_id", IntegerType(), False)
])

eventoDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(evento_schema).\
        load(path+"evento.csv")

eventoDF.show(4)

In [ ]:
deporte_schema = StructType([
          StructField("deporte_id",IntegerType(),False),
          StructField("deporte", StringType(),False)
])

deporteDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(deporte_schema).\
        load(path+"deporte.csv")

deporteDF.show(4)

In [ ]:
# juegos

juegos_schema = StructType([
    StructField('nombre_juego', StringType(),False),
    StructField('annio', IntegerType(),False),
    StructField('temporada', StringType(),False),
    StructField('ciudad', StringType(),False)
]    
)

juegosDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(juegos_schema).\
        load(path+"juegos.csv")

juegosDF.show(4)


 Clase 15 Operaciones sobre DF.
En esta clase aprendemos como obtener el schema de un DataFrame, a renombrar columnas y select().

In [ ]:
# Ver el esquema
deporteDF.printSchema()

In [ ]:
# Ver el esquema
deportistaDF.printSchema()

In [ ]:
#Renombrar columnas
deportistaOlimpicoDF = deportistaDF \
.withColumnRenamed("genero", 'sexo') \
.drop('altura')


In [ ]:
deportistaOlimpicoDF.printSchema()

In [ ]:
# Usando el rey SELECT
from pyspark.sql.functions import *
deportistaOlimpicoDF = deportistaOlimpicoDF.select("deportista_id","nombre",
 col("edad").alias("edadAlJugar"),
 "equipo_id"
 )

In [ ]:
deportistaOlimpicoDF.show(5)

In [ ]:
# Usando filter
deportistaOlimpicoDF = deportistaOlimpicoDF.filter(
    (deportistaOlimpicoDF.edadAlJugar != 0)
)

In [ ]:
deportistaOlimpicoDF.sort("edadAlJugar").show()

Clase 16 Agrupaciones y operaciones join sobre DF


In [ ]:
#vizualizamos los esquemas de los DF
deportistaOlimpicoDF.printSchema()

In [ ]:
resultadosDF.printSchema()

In [ ]:
juegosDF.printSchema()

In [ ]:
deporteDF.printSchema()

In [ ]:
paisesDF.printSchema()

In [ ]:
#JOINS
deportistaOlimpicoDF \
.join(
    resultadosDF,
    deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id,
    "left"
) \
.join(
    juegoDF,
    juegoDF.juego_id == resultadosDF.juego_id,
) \
.join(
    eventoDF,
    eventoDF.evento_id == resultadosDF.evento_id,
    "left"
) \
.select(deportistaOlimpicoDF.nombre,
        "edadAlJugar",
        "medalla",
        col("anio").alias("annio de juego"),
        deportistaOlimpicoDF.nombre.alias("Nombre disciplina")   
).show()



In [ ]:
#Reto hacer un JOIN con todas las medallas ganadoras
#unidas con el país y el equipo al que pertenecen estas medallas
resultadosDF \
.join(
    deportistaOlimpicoDF,
   resultadosDF.deportista_id == deportistaOlimpicoDF.deportista_id,
    "left"
) \
.join(
   paisesDF,
    deportistaOlimpicoDF.equipo_id == paisesDF.pais_id ,
    "left"
) \
.select("medalla",
        paisesDF.equipo,
       paisesDF.sigla
) \
.where (resultadosDF.medalla != "NA").show()




In [ ]:
# Clase 17 Solucion reto joins
resultadosDF.filter(resultadosDF.medalla != "NA") \
.join(
    deportistaOlimpicoDF,
     resultadosDF.deportista_id == deportistaOlimpicoDF.deportista_id,
     "left"     
) \
.join(
    paisesDF,
     deportistaOlimpicoDF.equipo_id == paisesDF.pais_id,
     "left"
) \
.select("medalla", paisesDF.equipo, paisesDF.sigla) \
.sort(col("sigla").desc()) \
.show()

In [ ]:
#Evento deportesOlimpicos
eventoSchema = StructType([
     structField('evento_id', IntegerType(), False),
     structField('nombre', StringType(), False),
     structField('deporte_id'), IntegerType(), False),
])
#Paso 2 Creo DataFrame con la lectura del Schema
deportesOlimpicosDF = sqlContext.read.schema(evntoSchema)\
            .option('header', 'true').csv(path+'evento.csv')

In [ ]:
Clase 18 Funciones de agrupación
En esta clase revisamos cuales son las funciones de agrupación con que cuenta Spark.

In [ ]:
#Clase 18 Funciones de agrupación
medallistaXanio = deportistaOlimpicoDF \
   .join (
       resultadosDF,
       deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id,
       "left"
         )\
   .join(
       juegoDF,
       juegoDF.juego_id == resultadosDF.juego_id,
       "left"
        )\
   .join(
     paisesDF,
     deportistaOlimpicoDF.equipo_id == paisesDF.pais_id,
     "left"
        )\
     .join(
         deportistaOlimpicoDF,
         deportistaOlimpicoDF.evento_id == resultadosDF.evento_id,
         "left"
        )\
      .join(
          deporteDF,
          deportistaOlimpicoDF.deporte_id == deporteDF.deporte_id,
          "left"
          )\
       .select(
           "sigla",
           "anio",
           "medalla",
           deportistaOlimpicoDF.nombre.alias("Nombre subdisciplina"),
           deporteDF.deporte.alias("Nombre disciplina"),
           deportistaOlimpicoDF.nombre
       )            
